In [1]:
import ipywidgets as widgets
_out = widgets.Output()
with _out:
    from ipynb.fs.full.invoiceDemoLibrary import *

In [2]:
assets = listInvoices(shop_owner_peer, user=shop_owner)[0]
_keys = calc_keys(assets)

# Demo Functions

In [3]:
def get_cert_cn(cert):
    return cert.split(',')[0].split('=')[1]
    #return cert.split('=')[1].split('@')[0]

def get_person_names_with_role(role,roles):
    result = []
    for person in roles:
        if role in person['roles']:
            #result.append(get_cert_cn(person['name']))
            result.append(person['name'])
    return result        
        
def get_person_certs_with_role(role,roles):
    result = []
    for person in roles:
        if role in person['roles']:
            result.append(person['name'])
    return result                     

In [4]:
def get_current_user_with_role(rolename):    
    if rolename == 'Buyer':
        return buyer
    elif  rolename == 'Seller':   
        return owner
    elif  rolename == 'Factor':   
        return factor
    elif  rolename == 'TaxInspector':    
        return taxinspector
    
def get_current_peer_with_role(rolename):    
    if rolename == 'Buyer':
        return peer_buyer
    elif  rolename == 'Seller':   
        return peer_owner
    elif  rolename == 'Factor':   
        return peer_factor
    elif  rolename == 'TaxInspector':    
        return peer_taxinspector    

def disable_all_fields():
    for key,value in _widget.container.fields.items():
        value.widget.disabled=True
        
def enable_fields_for_role(rolename):
    disable_all_fields()
    if rolename == 'Buyer':
        _widget.container.fields['received'].widget.disabled=False
        _widget.container.fields['receivedOrder'].widget.disabled=False

    elif  rolename == 'Seller':   
        _widget.container.fields['sold'].widget.disabled=False
        _widget.container.fields['claimPaid'].widget.disabled=False
        _widget.container.fields['claimPaidBy'].widget.disabled=False
        _widget.container.fields['owner'].widget.disabled=False
        
    elif  rolename == 'Factor':   
        #_widget.container.fields['sold'].widget.disabled=False
        _widget.container.fields['claimPaid'].widget.disabled=False
        _widget.container.fields['claimPaidBy'].widget.disabled=False
        #_widget.container.fields['owner'].widget.disabled=False
                
    elif  rolename == 'TaxInspector':    
        _widget.container.fields['taxReceived'].widget.disabled=False       

In [5]:
def create_dropdown_owner(_widget):
    _certs=[_widget.container.fields['owner'].widget.value]
    _certs = _certs + get_person_certs_with_role('Factor',roles)
    return widgets.Dropdown(options=_certs, value = _certs[0], description="owner", style=style, layout=layout)

def replace_owner_field_by_dropdown(_widget):
    _owners = create_dropdown_owner(_widget)
    _widget.container.fields['owner'] = Field(widget=_owners,path=_widget.container.fields['owner'].path)
    _widget.redisplay()

### role buyer

In [6]:
def init_fields_buyer(user=None,peer=None):
    #peer=get_peer_of_user(user)

    def received_changed(b):
        if b['name'] == 'value':
            receivedInvoice(peer, id=id, user=user)[1]

    def receivedOrder_changed(b):
        if b['name'] == 'value':
            receivedOrder(peer, id=id, user=user)[1]    

    _widget.container.fields['received'].widget.observe(received_changed)
    _widget.container.fields['receivedOrder'].widget.observe(receivedOrder_changed)
    enable_fields_for_role('Buyer')

### role owner / seller

In [7]:
def init_fields_seller(user=None,peer=None):
    #peer=get_peer_of_user(user)
    
    def sold_changed(b):
        if b['name'] == 'value':
            name=_widget.container.fields['owner'].widget.value
            transferInvoice(peer, id=id,name=name, user=user)[1]

    def claimPaid_changed(b):
        if b['name'] == 'value':
            receivedPayment(peer, id=id, payer= _widget.container.fields['claimPaidBy'].widget.value, user=user)

    enable_fields_for_role('Seller')    
    _widget.container.fields['sold'].widget.observe(sold_changed)
    _widget.container.fields['claimPaid'].widget.observe(claimPaid_changed)   
    replace_owner_field_by_dropdown(_widget)

### role factor (owner)

In [8]:
def init_fields_factor(user=None,peer=None):
    #peer=get_peer_of_user(user)
    
    def claimPaid_changed(b):
        if b['name'] == 'value':  
            receivedPayment(peer, id=id, payer= _widget.container.fields['claimPaidBy'].widget.value, user=user)

    enable_fields_for_role('Factor')    
    _widget.container.fields['claimPaid'].widget.observe(claimPaid_changed)   

### role tax inspector

In [9]:
def init_fields_taxinspector(user=None,peer=None):
    #peer=get_peer_of_user(user)

    def taxReceived_changed(b):
        if b['name'] == 'value':
            taxReceived(peer, id=id, user=user)

    _widget.container.fields['taxReceived'].widget.observe(taxReceived_changed)
    enable_fields_for_role('TaxInspector')

In [10]:
import base64

def get_height():
    global _out
    _out = widgets.Output()
    with _out:
        strlist = ['/mnt/conf/peer/block-operator.sh', 'info', '--channel', channel_name]
        cmd = toCmd(strlist)
        print(cmd)
        output = executePeerCmd(peers[peernames[0]].pod_name,  cmd)
        return json.loads(output)['height']

def uploadBlockToShare(id):
    strlist = ['/mnt/conf/peer/block-operator.sh fetch', '--channel', channel_name, '--block', str(id),'--publish', shared_fs_host]
    cmd = toCmd(strlist)
    output = executePeerCmd(peers[peernames[0]].pod_name,  cmd)
    
def getBlockFromShare(id):
    filename = 'invoicing-block-' + str(id) + '.block.json'
    downloadToLocal(host='10.15.136.41:30080',user='scray', pwd='scray', local_path='/tmp/',local_name=filename, remote_path='blocks/invoicing')
    output = subprocess.check_output(['cat', '/tmp/' + filename])
    return json.loads(output.decode('ascii'))           
    
def decode_block_invoice(block):
    _transaction = block['data']['data'][0]['payload']['data']['actions'][0]['payload']['action']['proposal_response_payload']['extension']['results']['ns_rwset'][1]['rwset']['writes'][0]
    if _transaction['value'] != None:
        return json.loads(base64.b64decode(_transaction['value']).decode("utf-8"))
    
def decode_block_transaction(block):
    _transaction = _block['data']['data'][0]['payload']['data']['actions'][0]['payload']['chaincode_proposal_payload']['input']['chaincode_spec']['input']['args']
    result = []
    for arg in _transaction:
        result.append(base64.b64decode(arg).decode("utf-8"))
    return result        

def get_transaction():
    with widgets.Output():
        height=get_height()
        uploadBlockToShare(height-1)    
        _block = getBlockFromShare(height-1)
        _invoice = decode_block_invoice(_block)
        _transaction = decode_block_transaction(_block)
        return _transaction,_invoice

In [11]:
#get_transaction()
#height=get_height()
#uploadBlockToShare(height-1)    
#_block = getBlockFromShare(height-1)
#_invoice = decode_block_invoice(_block)
#_transaction = decode_block_transaction(_block)

In [12]:
#_transaction
#get_transaction()

### Widgets

In [13]:
#invoice widget

#def asset_selected(x): 
#    global asset
#    asset = listInvoice(shop_owner_peer, id=assetOptions.value, user=shop_owner)[0]
    
#assetOptions = widgets.Dropdown(options=_keys)
#assetOptions.observe(asset_selected, names='value')

def widget_load_asset(widget,asset):
    widget.output.clear_output()
    widget.container = FieldContainer()
    with widget.output:
        widget.addChilds(asset,'/', hideUnused=True)

def reload_clicked(i):
    global buyer,peer_buyer
    global asset
    
    
    user = get_current_user_with_role(roleOptions.value)
    peer = get_current_peer_with_role(roleOptions.value)
    #print(user)
    #asset = listInvoice(shop_owner_peer, id, user=shop_owner)[0]
    asset = listInvoice(peer, id, user=user)[0]
    #print(asset['buyer'])
    with _out:
        buyer=get_cert_cn(asset['buyer'])
        peer_buyer=get_peer_of_user(buyer)
        
    widget_load_asset(_widget,asset)
    role_selected('')  

def role_selected(x): 
    if roleOptions.value == 'Buyer':
        init_fields_buyer(user=buyer,peer=peer_buyer)
    elif roleOptions.value == 'Seller':
        init_fields_seller(user=owner,peer=peer_owner)
    elif roleOptions.value == 'TaxInspector':    
        init_fields_taxinspector(user=taxinspector,peer=peer_taxinspector)
    elif roleOptions.value == 'Factor':    
        init_fields_factor(user=factor,peer=peer_factor)

def asset_selected(x): 
    global id
    id=assetOptions.value
    #asset = listInvoice(shop_owner_peer, id=assetOptions.value, user=shop_owner)[0]
    #roleOptions.value == 'Seller'
    reload_clicked('')
    
    
def _block_btn_reload_clicked(i):    
    height=get_height()
    _block_text.value = str(height)
    with _block_out:
        clear_output()  
        uploadBlockToShare(height-1)    
        _block = getBlockFromShare(height-1)
        _invoice = decode_block_invoice(_block)
        _transaction = decode_block_transaction(_block)
        _t = get_transaction()[0]
        clear_output()  
        facturx_widgets(_t,'/', hideUnused=True, hide=False).output
        
def create_invoice_widget():    
    #global _widget


    _widget = facturx_widgets(asset,'/', hideUnused=False, hide=True) 
    
    #display(_widget)

    HBox = widgets.HBox([_widget.output, options,assetOptions,block_explorer]) 
    display(HBox)

    assetOptions.value='asset14'
    
    return _widget

roleOptions = widgets.Dropdown(options=usedRoles)
#roleOptions.observe(role_selected, names='value')
roleOptions.observe(reload_clicked, names='value')
        
btn = widgets.Button(description=f'From Blockchain', disabled=False,button_style='success', icon='check')    
btn.on_click(reload_clicked)
#display (btn)

assetOptions = widgets.Dropdown(options=_keys)
assetOptions.observe(asset_selected, names='value')
options = widgets.VBox([roleOptions,btn])

_block_btn = widgets.Button(description=f'Explore Block', disabled=False,button_style='success', icon='check')    
_block_btn.on_click(_block_btn_reload_clicked)
_block_text = widgets.Text(description = 'height', style=style, layout=layout)
_block_out = widgets.Output()

block_explorer = widgets.VBox([_block_btn,_block_text,_block_out])

In [14]:
def role_selected(x): 
    if roleOptions.value == 'Buyer':
        init_fields_buyer(user=buyer,peer=peer_buyer)
    elif roleOptions.value == 'Seller':
        init_fields_seller(user=owner,peer=peer_owner)
    elif roleOptions.value == 'TaxInspector':    
        init_fields_taxinspector(user=taxinspector,peer=peer_taxinspector)
    elif roleOptions.value == 'Factor':    
        init_fields_factor(user=factor,peer=peer_factor)

In [15]:
_invoice = invoices.iloc[14]

In [16]:
def create_cluster_widget():
    output = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'get','service','hl-fabric-cc-external-invoice-net','-o','json'])
    _chaincode_service = json.loads(output.decode('ascii'))
    _chaincode_service_fields = {'name':_chaincode_service['metadata']['name'], 'nodePort':_chaincode_service['spec']['ports'][0]['nodePort']}

    _chaincode_widget = facturx_widgets(hideUnused=True, hide=True)
    #_header = _chaincode_widget.addHeader(text='Chaincode',bold=True, path='',hide=True )
    #_chaincode_widget.container.add(widget=_header)
    _chaincode_widget.append(data=_chaincode_service_fields)
    
    _sharepoint_widget = facturx_widgets({'sharePoint':shared_fs_host},'/', hideUnused=True, hide=True).output

    _peer_pods = create_pod_widget(_invoice)
    _orderer = ordererpeer(name='orderer-org1-scray-org',prefix='orderer').__dict__
    _orderer_pod = facturx_widgets(_orderer,'/', hideUnused=True, hide=True).output

    _column2 = widgets.VBox([_orderer_pod, _chaincode_widget.output, _sharepoint_widget])
    #_chaincode_widget.redisplay()
    return widgets.HBox([_peer_pods, _column2])


def display_role(roles, role_transctions, role,user,peer):
    _role_widget = facturx_widgets( hideUnused=True, hide=True, layout = {'width': '390px'}, style = {'description_width': '110px'})
    _header = _role_widget.addHeader(text='Rolle',bold=True, path='',hide=True )
    _role_widget.container.add(widget=_header)
    _role_widget.append(data=roles) 
    _header = _role_widget.addHeader(text='User',bold=True, path='',hide=True )
    _role_widget.container.add(widget=_header)
    _role_widget.append(data={'user':user,'peer':peer.name})
    
    _role_transctions_widget = facturx_widgets( hideUnused=True, hide=True, layout = {'width': '300px'}, style = {'description_width': '50px'})
    _header = _role_transctions_widget.addHeader(text='Transaktionen: ' + role,bold=True, path='',hide=True  )
    _role_transctions_widget.container.add(widget=_header)
    _role_transctions_widget.append(data=role_transctions)

    _hbox = widgets.HBox([_role_widget.output,_role_transctions_widget.output])
    display(_hbox)
    _role_widget.redisplay()
    _role_transctions_widget.redisplay()

In [17]:
def get_transactions_by_role(role):
    for _role in roleTransactions:
        if _role['role'] == role:
            return _role['transactions']

def update_role_widget(user=None,peer=None,role=None):
    _roles = getRoles(peer, user=user)[0]
    _role_transactions = get_transactions_by_role(role)
    
    if isinstance(_roles, str):
        _roles=''
    
    with roleWidget_out:
        clear_output()
        display_role(_roles, _role_transactions,role,user,peer)

def role_widget_selected(x): 
    if roleWidgetOptions.value == 'Buyer':
        update_role_widget(user=buyer,peer=peer_buyer,role=roleWidgetOptions.value)
    elif roleWidgetOptions.value == 'Seller':
        update_role_widget(user=owner,peer=peer_owner,role=roleWidgetOptions.value)
    elif roleWidgetOptions.value == 'TaxInspector':    
        update_role_widget(user=taxinspector,peer=peer_taxinspector,role=roleWidgetOptions.value)
    elif roleWidgetOptions.value == 'Factor':    
        update_role_widget(user=factor,peer=peer_factor,role=roleWidgetOptions.value)                     
        

roleWidgetOptions = widgets.Dropdown(options=usedRoles)
roleWidgetOptions.observe(role_widget_selected, names='value')     
roleWidget_out = widgets.Output()

# Blockchain (Peers, Orderer, Chaincode) und Sharepoint (WebDAV)

In [18]:
cluster_widget = create_cluster_widget()
display(cluster_widget)

# Rechnung aus Github-Repository (Factur-X / Parquet)

In [19]:
facturx_widgets(data=_invoice,path='/supplyChainTradeTransaction', hideUnused=True, hide=True).output

Output()

# Adressen

In [20]:
create_address_widget(_invoice)

# Rollen (werden in Blockchain gespeichert und sind änderbar)

In [21]:
display(roleWidgetOptions,roleWidget_out)    

Dropdown(options=('TaxInspector', 'Factor', 'Buyer', 'Seller'), value='TaxInspector')

Output()

# Invoice Widget

In [22]:
_widget = create_invoice_widget()
asset_selected('')

In [23]:
height=get_height()
_block_text.value = str(height)
uploadBlockToShare(height-1)    
_block = getBlockFromShare(height-1)
_invoice = decode_block_invoice(_block)
_transaction = decode_block_transaction(_block)
with _block_out:
    clear_output()  
    facturx_widgets(_transaction,'/', hideUnused=True, hide=False).output

/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-79666dbdd9-kvf9b -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer/block-operator.sh fetch --channel invoicing9 --block 18 --publish 10.14.128.38:30080


2022-01-05 10:44:57.897 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2022-01-05 10:44:57.910 UTC [cli.common] readBlock -> INFO 002 Received block: 18
2022-01-05 10:44:57.955 UTC [common.tools.configtxgen] main -> INFO 001 Loading configuration
2022-01-05 10:44:57.956 UTC [common.tools.configtxgen] doInspectBlock -> INFO 002 Inspecting block
2022-01-05 10:44:57.956 UTC [common.tools.configtxgen] doInspectBlock -> INFO 003 Parsing genesis block
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   230  100   230    0     0   9200      0 --:--:-- --:--:-- --:--:--  9200
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   193  100   193    0     0   8772      0 --:--:-- --:--:-- --:--:--  8772
  % Total    % Received % Xferd  A

In [24]:
_transaction, _invoice

(['ReceivedPayment', 'asset17', ''],
 {'ID': 'asset17',
  'owner': 'x509::CN=Tony.Dubois@aubonmoulin.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.aubonmoulin.kubernetes.research.dev.seeburger.de,O=aubonmoulin.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE',
  'buyer': 'x509::CN=Alexandre.Payet@majolieboutique.net,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.aubonmoulin.kubernetes.research.dev.seeburger.de,O=aubonmoulin.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE',
  'hash': 6265861722676072020,
  'invoiceNumber': 'AV-2017-0005',
  'tax': -14.99,
  'netto': -218.48,
  'countryOrigin': 'FR',
  'countryBuyer': 'FR',
  'status': '',
  'received': True,
  'receivedOrder': True,
  'sold': False,
  'claimPaid': True,
  'claimPaidBy': '',
  'taxExemptionReason': '',
  'taxReceived': False})

In [25]:
with _block_out:
    clear_output()  

height=get_height()
_block_text.value = str(height)    